In [2]:
import numpy as np
import pandas as pd
import scipy
import skbio

In [3]:
def MINT(file, method):
        df = pd.read_csv(file)
        id  = df.ix[:,0]
        otu = df.ix[:,2:]

        u_obs = df.ix[:,0:2].drop_duplicates(df.columns[0])  # group information for each subject
        id_set = u_obs.ix[:,0]
        grp = u_obs.ix[:,1]
        id_n = len(id_set)

        ## Caculate Kendall correlation for temporal interdependence
        res = list()
        for i, id_key in enumerate( id_set ) :
            otu_id = otu.loc[ id == id_key ]
            res.append( otu_id.corr(method = method).fillna(0) )

        ## Caculate sample distance for temporal interdependence 
        dist = np.zeros( (id_n, id_n) )
        for i, id1_key in enumerate( id_set ) :
            for j, id1_key in enumerate( id_set ):
                dist[i,j] = scipy.linalg.norm(res[i] - res[j])  # F-norm

        ## Perform Permutation Manova Test
        test = skbio.stats.distance.permanova(skbio.DistanceMatrix(dist), grp)
        return({"test": test, "dist": dist, "meta": u_obs, "status":1})

method = "kendall"
file   = "mice.csv"
res = MINT(file, method)

In [4]:
res["test"]

method name               PERMANOVA
test statistic name        pseudo-F
sample size                      20
number of groups                  2
test statistic              3.02628
p-value                       0.001
number of permutations          999
Name: PERMANOVA results, dtype: object